Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook 
for the following questions.  

QUESTIONS:



In [1]:
from sqlalchemy import create_engine
import os
os.getcwd()
engine = create_engine('sqlite:///sqlite_db.db')
print(engine.table_names())
con = engine.connect()
#rs = con.execute('Select * from Members')

#con.close()

['Bookings', 'Facilities', 'Members']


/* Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */

In [2]:
rs = con.execute('''
SELECT sub.facility, sum(cost) as totalrev
from (
SELECT f.name as facility,
case 
when m.memid = 0 then f.guestcost
else f.membercost 
end as cost
FROM Bookings as b
inner join Members as m
on m.memid = b.memid 
inner join Facilities as f
on f.facid = b.facid
order by cost desc 
) as sub
group by facility
having sum(cost)<1000
order by totalrev
                 ''')

import pandas as pd

pd.DataFrame(rs.fetchall(), columns = rs.keys())

,facility,totalrev
0,Table Tennis,90.0
1,Snooker Table,115.0
2,Pool Table,265.0
3,Badminton Court,604.5




/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */





In [11]:
rs = engine.execute("""
select (M1.firstname || ' '|| M1.surname) as member, 
M1.recommendedby, 
(M2.firstname || ' '|| M2.surname) as recname
from  Members as M1
INNER join Members as M2
on M1.recommendedby = M2.memid
Where M1.recommendedby IS NOT NULL OR recname NOT LIKE 'None'
ORDER BY member
""")

pd.DataFrame(rs.fetchall(), columns = rs.keys())

,member,recommendedby,recname
0,Anna Mackenzie,1,Darren Smith
1,Anne Baker,9,Ponder Stibbons
2,Charles Owen,1,Darren Smith
3,David Jones,4,Janice Joplette
4,David Pinker,13,Jemima Farrell
5,Douglas Jones,11,David Jones
6,Erica Crumpet,2,Tracy Smith
7,Florence Bader,9,Ponder Stibbons
8,Gerald Butters,1,Darren Smith
9,Henrietta Rumney,20,Matthew Genting


/* Q12: Find the facilities with their usage by member, but not guests */




In [17]:
rs = engine.execute("""
Select name, count(*) as memberusage
from Facilities as f
inner join Bookings as b
on b.facid = f.facid
where b.facid != 0
group by name
""")

pd.DataFrame(rs.fetchall(), columns = rs.keys())

,name,memberusage
0,Badminton Court,383
1,Massage Room 1,629
2,Massage Room 2,111
3,Pool Table,836
4,Snooker Table,444
5,Squash Court,440
6,Table Tennis,403
7,Tennis Court 2,389


/* Q13: Find the facilities usage by month, but not guests */

In [19]:

rs = engine.execute("""
Select name, count(*) as memberusage, strftime('%m',b.starttime) as month
from Facilities as f
inner join Bookings as b
on b.facid = f.facid
where b.facid != 0
group by name, month
""")

pd.DataFrame(rs.fetchall(), columns = rs.keys())

,name,memberusage,month
0,Badminton Court,56,07
1,Badminton Court,146,08
2,Badminton Court,181,09
3,Massage Room 1,123,07
4,Massage Room 1,224,08
5,Massage Room 1,282,09
6,Massage Room 2,12,07
7,Massage Room 2,40,08
8,Massage Room 2,59,09
9,Pool Table,110,07
